## Прикладная статистика в машинном обучении

### Семинар 11: Bootstrap

#### Идея.

Пусть на основе выборки $X_1,$ $\ldots$, $X_n$ мы каким-то способом получили точечную оценку неизвестного параметра $\theta$ и обозначили её $\hat{\theta}$. Теперь наша цель состоит в том, чтобы построить асимптотический доверительный интервал для $\theta$. Bootstrap предлагает оценить дисперсию $\hat{\theta}$ на основе тех же данных, на которых была получена сама оценка $\hat{\theta}$.

> "Pull oneself up by one's bootstraps": *(idiomatic) To succeed only by one's own efforts or abilities.* (Wiktionary)

---

#### Построение выборки (Sampling).

Вспомним, что *построением выборки (sampling)* называется выбор элементов генеральной совокупности (какого-то множетства или распределения). Случайная выборка строится путём случайного выбора наблюдений. Случайную выборку можно строить двумя способами:
- без возвращения (without replacement, simple random sampling),
- с возвращением (with replacement).

**Вопросы на подумать:**
1. Можно ли рассматривать построение выборки с возвращением из множества $\{1, 2, \ldots, 8\}$ как процедуру последовательного подбрасывания восьмигранной кости?
2. Когда на практике построение выборки с возвращением и без возвращения эквивалентны?

---

#### Эмпирическая функция распределения.

Также вспомним, что на основе выборки можно построить оценку функции распределения, из которого были взяты наблюдения. Для этого для каждого значения $x_i$ рассчитывается его доля в выборке.

---

#### Построение boostrap-выборки (Resampling).

*Построением boostrap-выборки (resampling)* называется **построение выборки с возвращением из эмпирического распределения**.

**Пример 1:** пусть имеется набор чисел $1$, $1$, $2$, $3$, $10$, $11$, $11$, который рассматривается как выборка, взятая из какого-то распределения. Чтобы построить выборку с возвращением из эмпирического распределения, мы должны сложить эти числа в шляпу и не глядя вытаскивать их одно за другим, записывая результат и возвращая число обратно.

**Формально:** пусть дана выборка $X_1$, $\ldots$, $X_n$. Построением boostrap-выборки называется выбор номера $i$ из равновероятного на $\{1, 2, \ldots n\}$ распределения и взятие $X_i$ как одного значения этой выборки.

**Продолжение примера 1:** пусть мы хотим построить boostrap-выборку размера 3 для чисел из примера 1. Тогда мы можем подбросить семигранную (так как семь наблюдений) кость три раза и, например, получить:
$$
3, 3, 7, 1
$$
Тогда boostrap-выборка будет:
$$
2, 2, 11, 1
$$

Будем обозначать boostrap-выборку как $X_1^*$, $\ldots$, $X_m^*$.

---

#### Эмпирический bootstrap.

*Выборкой эмпирического bootstrap* нызвается boostrap-выборка того же размера, что и оригинальная выборка:

для выборки $X_1$, $\ldots$, $X_n$ это $X_1^*$, $\ldots$, $X_n^*$. Тогда boostrap говорит, что $F \approx F^*$, а дисперсия статистики, рассчитанной на основе выборки $X$, примерно равна дисперсии статистики, рассчитанной на основе $X^*$.

## Зачем это нужно?

У нас есть выборка, нам кажется, что наблюдаемые значения распределены нормально, но p-value гипотезы о принадлежности наблюдаемой выборки нормальному распределению равно 0.049. Чтобы не лукавить при использовании теорем, основанных на принадлежности данного распределения нормальному, не будем принимать гипотезу и оцениваем требуемые доверительные параметров с помощью бутстрапирования, а не традиционных формул.

#### 1. Reverse bootstrap percentile.

**Пример 2:** пусть дана выборка из некоторого распределения с конечным матожиданием $\mu$:

In [ ]:
X = [30, 37, 36, 43, 42, 43, 43, 46, 41, 42]

Постройте 80% доверительный интервал для $\mu$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
## Шаг 1: Найдите точечную оценку неизвестного параметра
# (ML)
mu_hat = np.mean(X)
mu_hat

40.3

Мы бы хотели найти распределение величины $d = \hat{X} - \mu$, потому что тогда мы можем построить доверительный интервал для $\mu$ как
$$
P(d_{0.9} \le \bar{X} - \mu \le d_{0.1}) = 0.8 \Rightarrow \mu \in [\bar{x} - d_{0.1}; \bar{x} - d_{0.9}].
$$

Boostrap говорит, что распределение $d$ можно приблизить распределением $d^* = \bar{x^*} - \bar{x}$.

Удача заключается в том, что можно сгенерировать $d^*$ сколько угодно раз, а потому возможно получить достаточно точную оценку распределения $d^*$.

In [ ]:
## Шаг 2: Сгенерируйте 20 bootstrap-выборок из X и сохраните их в матрицу n x 20
np.random.seed(1234)
bootstrap_samples = np.random.choice(X, size = (len(X), 2000), replace = True)
bootstrap_samples

array([[43, 43, 43, ..., 46, 42, 46],
       [30, 42, 37, ..., 41, 46, 43],
       [41, 42, 30, ..., 41, 43, 43],
       ...,
       [43, 43, 43, ..., 41, 43, 41],
       [36, 30, 42, ..., 43, 30, 42],
       [41, 30, 46, ..., 37, 37, 42]])

In [ ]:
# Шаг 3: Рассчитайте выборочное среднее по каждой bootstrap-выборке
bootstrap_means = np.mean(bootstrap_samples, axis = 0)
bootstrap_means

array([38.9, 38.7, 39.6, ..., 40.2, 40.7, 42.8])

In [ ]:
# Шаг 4: Рассчитайте d*
d_star = bootstrap_means - mu_hat
d_star

array([-1.4, -1.6, -0.7, ..., -0.1,  0.4,  2.5])

In [ ]:
d_star_quant = np.quantile(d_star, (0.1, 0.9))
d_star_quant

array([-1.9,  1.7])

In [ ]:
# Шаг 5: Найдите 0.1 и 0.9 квантили для d_star
d_star_quant = np.quantile(d_star, (0.1, 0.9))
d_star_quant

array([-2. ,  1.7])

In [ ]:
# Шаг 6: Постройте 80%-ый доверительный интервал для mu
LB = mu_hat - d_star_quant[1]
UB = mu_hat - d_star_quant[0]
print('[', LB, ';', UB, ']')

[ 38.599999999999994 ; 42.3 ]


**Пример 3:** для той же выборки постройте 90%-ый доверительный интервал для медианы.

In [ ]:
from seaborn.algorithms import bootstrap
## Шаг 1: Найдите точечную оценку неизвестного параметра
med = np.median(X)
print(med)
## Шаг 2: Сгенерируйте 100 boostrap-выборок из X и сохраните их в матрицу n x 100
bootstrap_samples = np.random.choice(X, size = (len(X), 10000), replace = True)
# Шаг 3: Рассчитайте выборочную медиану по каждой bootstrap-выборке
bootstrap_medians = np.median(bootstrap_samples, axis = 0)
# Шаг 4: Рассчитайте d*
d_star = bootstrap_medians - med
print(d_star)
# Шаг 5: Найдите 0.05 и 0.95 квантили для d_star
d_star_quant = np.quantile(d_star, (0.05, 0.95))
print(d_star_quant)
# Шаг 6: Постройте 90%-ый доверительный интервал для медианы
LB = med - d_star_quant[1]
UB = med - d_star_quant[0]
print('[', LB, ';', UB, ']')

42.0
[ 0.   1.   1.  ...  0.   0.5 -1. ]
[-3.5  1. ]
[ 41.0 ; 45.5 ]


#### 2. t-bootstrap

**Идея:** сгенерировать не $d^*$, а 
$$
t^* = \dfrac{\hat{\theta^*} - \hat{\theta}}{se^{*}}
$$

**Пример 4:** для той же выборки постройте 80%-ый доверительный интервал для $\mu$.

In [ ]:
## Шаг 1: Найдите точечную оценку неизвестного параметра
x_hat = np.mean(X)
## Шаг 2: Сгенерируйте 100 boostrap-выборок из X и сохраните их в матрицу n x 100
bootstrap_samples = np.random.choice(X, size = (len(X), 100), replace = True)
# Шаг 3: Рассчитайте выборочное среднее по каждой bootstrap-выборке
bootstrap_means = np.mean(bootstrap_samples, axis = 0)
# Шаг 4: Рассчитайте стандартную ошибку по каждой bootstrap-выборке
bootstrap_ses = np.std(bootstrap_samples, axis = 0)
# Шаг 5: Рассчитайте t*
t_star = (bootstrap_means - x_hat) / bootstrap_ses
# Шаг 6: Найдите 0.1 и 0.9 квантили для t_star
t_star_quant = np.quantile(t_star, (0.1, 0.9))
# Шаг 7: Постройте 80%-ый доверительный интервал для mu
LB = x_hat - t_star_quant[1] * np.std(X)
UB = x_hat - t_star_quant[0] * np.std(X)
print('[', LB, ';', UB, ']')

[ 36.785933431263096 ; 42.156453689875775 ]


А как считать t-bootstrap, если оцениваем медиану? Что использовать как стандартное отклонение?

#### 3. Bootstrap percentile method

**!Danger:** не используйте этот метод.

**Идея:** использовать распределение не $d^*$, а распределение $\hat{\theta^*}$.

**Пример 5:** для той же выборки постройте 80%-ый доверительный интервал для $\mu$.

In [ ]:
## Шаг 1: Сгенерируйте 100 boostrap-выборок из X и сохраните их в матрицу n x 100
bootstrap_samples = np.random.choice(X, size = (len(X), 100), replace = True)
# Шаг 2: Рассчитайте выборочное среднее по каждой bootstrap-выборке
bootstrap_means = np.mean(bootstrap_samples, axis = 0)
# Шаг 3: Найдите 0.1 и 0.9 квантили для bootstrap_means
quant = np.quantile(bootstrap_means, (0.1, 0.9))
# Шаг 4: Постройте 80%-ый доверительный интервал для mu
LB = quant[0]
UB = quant[1]
print('[', LB, ';', UB, ']')

[ 38.7 ; 42.01 ]


**Пример 6:** покажите, насколько хорошо различные эмпирические доверительные интервалы накрывают истинное значение параметра.

In [ ]:
rbp_up = 0
tb = 0
bpm = 0

for i in range(100000):
    Y = np.random.normal(4, 0.25, 100)
    
    # RBP
    mu_hat = np.mean(Y)
    bootstrap_samples = np.random.choice(Y, size = (len(Y), 100), replace = True)
    bootstrap_means = np.mean(bootstrap_samples, axis = 0)
    d_star = bootstrap_means - mu_hat
    d_star_quant = np.quantile(d_star, (0.1, 0.9))
    LB = mu_hat - d_star_quant[1]
    UB = mu_hat - d_star_quant[0]
    
    if (LB <= 4) & (UB >= 4):
        rbp_up += 1
    
    # TB
    x_hat = np.mean(Y)
    bootstrap_samples = np.random.choice(Y, size = (len(Y), 100), replace = True)
    bootstrap_means = np.mean(bootstrap_samples, axis = 0)
    bootstrap_ses = np.std(bootstrap_samples, axis = 0)
    t_star = (bootstrap_means - x_hat) / bootstrap_ses
    t_star_quant = np.quantile(t_star, (0.1, 0.9))
    LB = x_hat - t_star_quant[1] * np.std(Y)
    UB = x_hat - t_star_quant[0] * np.std(Y)
    
    if (LB <= 4) & (UB >= 4):
        tb += 1
        
    # BP
    bootstrap_samples = np.random.choice(Y, size = (len(Y), 100), replace = True)
    bootstrap_means = np.mean(bootstrap_samples, axis = 0)
    quant = np.quantile(bootstrap_means, (0.1, 0.9))
    LB = quant[0]
    UB = quant[1]
    if (LB <= 4) & (UB >= 4):
        bpm += 1

In [ ]:
rbp_up / 1000

77.936

In [ ]:
tb / 1000

78.473

In [ ]:
bpm / 1000

77.963

---

#### Параметрический Bootstrap.

Совпадает с reverse bootstrap percentile за исключением того, что bootstrap-выборка набирается из параметризованного распределения.

**Пример 7:** предположим, что $X_1$, $\ldots$, $X_{300}$ -- случайная выборка из экспоненциального exp$(\lambda)$ распределения. Пусть оказалось так, что $\bar{X} = 2$. Постройте 95%-ый доверительный интервал для $\lambda$.

In [ ]:
## Шаг 1: Найдите точечную оценку неизвестного параметра
lambda_hat = 1 / 2
## Шаг 2: Сгенерируйте 1000 boostrap-выборок из X и сохраните их в матрицу n x 1000
bootstrap_samples = np.random.exponential(1/lambda_hat, (300, 100000))
# Шаг 3: Рассчитайте оценку lambda по каждой bootstrap-выборке
lambda_star = 1 / np.mean(bootstrap_samples, axis = 0)
# Шаг 4: Рассчитайте d*
d_star = lambda_star - lambda_hat
# Шаг 5: Найдите 0.025 и 0.975 квантили для d_star
d_star_quant = np.quantile(d_star, (0.025, 0.975))
# Шаг 6: Постройте 95%-ый доверительный интервал для lambda
LB = lambda_hat - d_star_quant[1]
UB = lambda_hat - d_star_quant[0]
print('[', LB, ';', UB, ']')

[ 0.4379896541827353 ; 0.552132520316605 ]


In [ ]:
## Шаг 1: Найдите точечную оценку неизвестного параметра
lambda_hat = 1 / 2
## Шаг 2: Сгенерируйте 1000 boostrap-выборок из X и сохраните их в матрицу n x 1000
bootstrap_samples = np.random.exponential(1/lambda_hat, (300, 100000))
# Шаг 3: Рассчитайте оценку lambda по каждой bootstrap-выборке
lambda_star = 1 / np.mean(bootstrap_samples, axis = 0)
# Шаг 4: Рассчитайте d*
#d_star = lambda_star - lambda_hat
# Шаг 5: Найдите 0.025 и 0.975 квантили для d_star
d_star_quant = np.quantile(lambda_star, (0.025, 0.975))
# Шаг 6: Постройте 95%-ый доверительный интервал для lambda
LB = d_star_quant[0]
UB = d_star_quant[1]
print('[', LB, ';', UB, ']')

[ 0.4478392474110142 ; 0.5615486362525398 ]


---

# Примеры практического применения

Оценка статистик, для которых нет доверительных интервалов в явном виде (минимумы, максимумы, медианы...):
0. Есть модель регрессии (Доходность проектов от заказчиков) - к нам пришёл Газпром, но у нас градиентный бустинг (не может предсказать то, что не видела никогда). 
1. Вы мониторите фичи модели, в которую на прод ежедневно поступают новые сэмплы для скоринга, хочется понимать, насколько адекватные значения прилетают
2. Как оценить доверительный интервал, чтобы понять, в каких случаях следует сигналить о превышении приходящими фичами критических значений?
3. Бутстрапируем! Берём подвыборки из обучающей выборки размерности, равной средней ежедневной


$N = 10^6$ сэмплов

$n^* = 1000$ сэмплов

$n_{iter} = 10000$ итераций

Считаем на подвыборках размерности $n^*$ максимумы и у нас получилось 10000 разных максимумов, которые как-то распределены. Мы оцениваем доверительный интервал, при котором не выводим сигнал начальству о критической ошибки, к которым не готова модель машинного обучения. 

За счёт бутстрапа можем с вероятностью 99% заверять начальство в том, что максимальный возраст подающих заявки клиентов в диапазоне [60, 75]. 



#### Источники мудрости:
[1] [Tim Hesterberg "What Teachers Should Know about the Bootstrap:Resampling in the Undergraduate StatisticsCurriculum?"](https://arxiv.org/pdf/1411.5279.pdf)

[2] [Jeremy Orloff, Jonathan Bloom "Bootstrap Confidence Intervals"](https://ocw.mit.edu/courses/mathematics/18-05-introduction-to-probability-and-statistics-spring-2014/readings/MIT18_05S14_Reading24.pdf)

[3] [Английская википедия](https://en.wikipedia.org/wiki/Bootstrapping_(statistics))